# Lab 12-5 sequence to sequence (Keras + eager version)

### simple neural machine translation training

* sequence to sequence
  
### Reference
* [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)

In [1]:
pip install tensorflow==1.15

  Found existing installation: setuptools 40.8.0
    Uninstalling setuptools-40.8.0:
      Successfully uninstalled setuptools-40.8.0
Note: you may need to restart the kernel to use updated packages.


In [142]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os
import pandas as pd
import re
#요약
from gensim.summarization.summarizer import summarize




1.15.0


In [264]:

#path = '/content/drive/My Drive/Colab Notebooks/moviedata2.json'
df = pd.read_json('moviedata_preprocessed.json')


# 전처리

In [269]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

example = "Family is not an important thing. It's everything."
stop_words = set(stopwords.words('english')) 

word_tokens = word_tokenize(example)

result = []
for w in word_tokens: 
    if w not in stop_words: 
        result.append(w) 

print(word_tokens) 
print(result) 

['Family', 'is', 'not', 'an', 'important', 'thing', '.', 'It', "'s", 'everything', '.']
['Family', 'important', 'thing', '.', 'It', "'s", 'everything', '.']


In [265]:
#df = movie_dataframe

df.movie_name = df.movie_name.apply(lambda x: x.replace('혻',''))

df.movie_summary = df.movie_summary.apply(lambda x: str(x).lower())
df.movie_summary = df.movie_summary.apply(lambda x: re.sub("[^A-Za-z. ]", '', x))
df.movie_summary = df.movie_summary.apply(lambda x: x.replace('\\n',''))

#df.movie_synopsis = df.movie_synopsis.apply(lambda x : str(x).lower())
#df.movie_synopsis = df.movie_synopsis.apply(lambda x : re.sub('[^A-Za-z. ]', '', x))

df.movie_tagline = df.movie_tagline.apply(lambda x: str(x).lower())
df.movie_tagline = df.movie_tagline.apply(lambda x: x.replace('\\n','').replace('\\xa0',''))
df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub('[^A-Za-z. ]', '', x))
df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub(' {2,}', '', x))

#태그라인 없는 데이터 row 삭제
index = df[df['movie_tagline']==''].index
df.drop(index, inplace = True)

#summary 없는 데이터 row 삭제
summary_index = df[df['movie_summary']==''].index
df.drop(summary_index, inplace = True)

index = df[df['movie_summary'].str.contains('  ')].index
df.drop(index , inplace = True)




In [274]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
df['shrinked'] = ""
#example = "Family is not an important thing. It's everything."
stop_words = set(stopwords.words('english')) 
for idx, i in enumerate(df.movie_summary[:1000]):
    word_tokens = word_tokenize(i)

    result = []
    for w in word_tokens: 
        if w not in stop_words: 
            result.append(w) 
    df['shrinked'][idx] = result
print(word_tokens) 
print(result) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


['hobsonvilles', 'citizens', 'hire', 'a', 'gunslinger', 'to', 'rid', 'them', 'of', 'the', 'mcintock', 'clan', 'that', 'is', 'forcibly', 'and', 'unlawfully', 'taxing', 'the', 'townsfolk', 'under', 'the', 'pretext', 'of', 'town', 'development', '.', 'master', 'gunslinger', 'sabata', 'arrives', 'in', 'hobsonville', 'a', 'town', 'completely', 'owned', 'by', 'mcintock', 'a', 'robber', 'baron', 'who', 'is', 'taxing', 'the', 'inhabitants', 'for', 'the', 'cost', 'of', 'future', 'improvements', 'to', 'the', 'town', '.', 'or', 'thats', 'what', 'mcintock', 'says', 'hell', 'do', 'with', 'the', 'money', '...']
['hobsonvilles', 'citizens', 'hire', 'gunslinger', 'rid', 'mcintock', 'clan', 'forcibly', 'unlawfully', 'taxing', 'townsfolk', 'pretext', 'town', 'development', '.', 'master', 'gunslinger', 'sabata', 'arrives', 'hobsonville', 'town', 'completely', 'owned', 'mcintock', 'robber', 'baron', 'taxing', 'inhabitants', 'cost', 'future', 'improvements', 'town', '.', 'thats', 'mcintock', 'says', 'hell'

In [266]:
df= df.reset_index()

In [276]:
df = df[:1000]

In [277]:
df

,level_0,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline,shrinked
0,1,2,"[Horror, Sci-Fi]",Alien,www.imdb.com/title/tt0078748/mediaviewer/rm299...,after a space merchant vessel perceives an unk...,[The opening credits appear in front of a larg...,its alien the th passenger.,"[space, merchant, vessel, perceives, unknown, ..."
1,10,3690,"[Drama, Thriller, War]",Innocent Voices,www.imdb.com/title/tt0387914/mediaviewer/rm103...,a young boy in an effort to have a normal chil...,"[\n , \n ]",how much will a mother sacrifice for the love ...,"[young, boy, effort, normal, childhood, el, sa..."
2,100,3848,"[Comedy, Crime]","Torrente, el brazo tonto de la ley",www.imdb.com/title/tt0120868/mediaviewer/rm603...,jos luis torrente a former policeman fights ag...,"[\n , \n ]",when you thought that spanish cinema was impro...,"[jos, luis, torrente, former, policeman, fight..."
3,1001,1871,"[Biography, Crime, Drama, Thriller]",The Falcon and the Snowman,www.imdb.com/title/tt0087231/mediaviewer/rm359...,the true story of a disillusioned military con...,"[Christopher Boyce (Timothy Hutton), a young s...",the falconthe snowman. a true story.,"[true, story, disillusioned, military, contrac..."
4,1002,2168,"[Comedy, Romance]",Waking Up in Reno,www.imdb.com/title/tt0219400/mediaviewer/rm104...,a romantic comedy about two trashy couples tra...,"[\n , \n ]",this outrageous road trip really heats up,"[romantic, comedy, two, trashy, couples, trave..."
5,1003,5631,[Comedy],Beyond Therapy,www.imdb.com/title/tt0092646/mediaviewer/rm974...,this is an insane and fastpaced romantic comed...,"[\n , \n ]",comic relief for incurable romantics.,"[insane, fastpaced, romantic, comedy, bizarre,..."
6,1004,5638,"[War, Western]",Springfield Rifle,www.imdb.com/title/tt0045184/mediaviewer/rm438...,major lex kearney becomes the norths first cou...,"[\n , \n ]",the gun... the girl... they made one man the e...,"[major, lex, kearney, becomes, norths, first, ..."
7,1005,507,"[Action, Adventure, Sci-Fi, Thriller]",Hardcore Henry,www.imdb.com/title/tt3072482/mediaviewer/rm625...,henry is resurrected from death with no memory...,"[Note: this entire, experimental film takes pl...",first they made him dangerous. then they made ...,"[henry, resurrected, death, memory, must, save..."
8,1006,5640,"[Family, Fantasy]",Little Ghost,www.imdb.com/title/tt0119550/mediaviewer/rm104...,while on location in a spooky romanian villa w...,"[\n , \n ]",its always an adventure when your best friend ...,"[location, spooky, romanian, villa, hollywood,..."
9,1007,5641,"[Comedy, Drama, Sci-Fi]",King Lear,www.imdb.com/title/tt0093349/mediaviewer/rm141...,everything returns to normal after chernobyl. ...,[Centering around the word No Thing . Jean Luc...,a picture shot in the back.,"[everything, returns, normal, chernobyl, ., ev..."


In [282]:

tl = list(df["movie_tagline"])
tl = ["".join([ j for j in i if j != "\n"]).strip() for i in tl]

summ = list(df['movie_summary'])

df.movie_tagline = tl
df.movie_summary = summ

sources = [idx for idx in df['shrinked']]
targets = [i.split() for i in df['movie_tagline']]


In [283]:
sources

[['space',
  'merchant',
  'vessel',
  'perceives',
  'unknown',
  'transmission',
  'distress',
  'call',
  'landing',
  'source',
  'moon',
  'finds',
  'one',
  'crew',
  'attacked',
  'mysterious',
  'lifeform',
  'soon',
  'realize',
  'life',
  'cycle',
  'merely',
  'begun',
  '.',
  'distant',
  'future',
  'crew',
  'commercial',
  'spaceship',
  'nostromo',
  'way',
  'home',
  'pick',
  'distress',
  'call',
  'distant',
  'moon',
  '.',
  'crew',
  'obligation',
  'investigate',
  'spaceship',
  'descends',
  'moon',
  'afterwards',
  '.',
  'rough',
  'landing',
  'three',
  'crew',
  'members',
  'leave',
  'spaceship',
  'explore',
  'area',
  'moon',
  '.',
  'time',
  'discover',
  'hive',
  'colony',
  'unknown',
  'creature',
  'ships',
  'computer',
  'deciphers',
  'message',
  'warning',
  'distress',
  'call',
  '.',
  'one',
  'eggs',
  'disturbed',
  'crew',
  'realizes',
  'alone',
  'spaceship',
  'must',
  'deal',
  'consequences',
  '.',
  'far',
  'reaches

In [263]:
df.movie_summary[0]

'four teen girls diving in a ruined underwater city quickly learn theyve entered the territory of the deadliest shark species in the claustrophobic labyrinth of submerged caves. 47 meters down uncaged follows the diving adventure of four teenage girls sophie nlisse corinne foxx brianne tju and sistine stallone exploring a submerged mayan city. once inside their rush of excitement turns into a jolt of terror as they discover the sunken ruins are a hunting ground for deadly great white sharks. with their air supply steadily dwindling the friends must navigate the underwater labyrinth of claustrophobic caves and eerie tunnels in search of a way out of their watery hell.'

# 단어 나누기

In [284]:

# vocabulary for sources
s_vocab = list(set(sum(sources, [])))
s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}



In [285]:
# vocabulary for targets
t_vocab = list(set(sum(targets, [])))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>', '<eos>'] + t_vocab
target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}




In [286]:
def preprocess(sequences, max_len, dic, mode = 'source'):
    assert mode in ['source', 'target'], 'source와 target 중에 선택해주세요.'
    
    if mode == 'source':
        # preprocessing for source (encoder)
        s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
        
        #s_input = [[x for x in each_input if x] for each_input in s_input]
        #pprint(s_input)
        s_len = list(map(lambda sentence : len(sentence), s_input))
        s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
        return s_len, s_input
    

    elif mode == 'target':
        # preprocessing for target (decoder)
        # input
        t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
        
        #t_input = [[x for x in each_input if x] for each_input in t_input]
        #pprint(t_input)
        t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
        t_len = list(map(lambda sentence : len(sentence), t_input))
        t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')
     
        # output
        t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
        t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
        t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')
        
        return t_len, t_input, t_output

In [290]:
sum_ = 0
for i in df.movie_summary:
    length = len(i.split())
    sum_ += length
average_no = sum_ / len(df) #summary당 평균 122.84 단어
# preprocessing for source
#단어 수는 120개로 제한
s_max_len = 60
s_len, s_input = preprocess(sequences = sources,
                            max_len = s_max_len, dic = source2idx, mode = 'source')
print(s_max_len)

60


In [291]:
sum_ = 0
for i in df.movie_tagline:
    length = len(i.split())
    sum_ += length
average_no = sum_ / len(df) #각 tagline별 단어 수는 9.6개

# preprocessing for target
#단어 수 10개로 제한
t_max_len = int(average_no)
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print(t_max_len)
#print(t_len, t_input, t_output)

9


# hyper-param

In [299]:
# hyper-parameters
epochs = 100
batch_size = 4
learning_rate = .005
total_step = epochs / batch_size
buffer_size = 100
n_batch = buffer_size//batch_size
embedding_dim = 32
units =64

# input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output))
data = data.shuffle(buffer_size = buffer_size)
data = data.batch(batch_size = batch_size)
# s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()

In [300]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [301]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [302]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
                
    def call(self, x, hidden, enc_output):
        
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [303]:
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    
#     print("real: {}".format(real))
#     print("pred: {}".format(pred))
#     print("mask: {}".format(mask))
#     print("loss: {}".format(tf.reduce_mean(loss_)))
    
    return tf.reduce_mean(loss_)

# creating optimizer
optimizer = tf.train.AdamOptimizer()

# creating check point (Object-based saving)
checkpoint_dir = './data_out/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                encoder=encoder,
                                decoder=decoder)
print(checkpoint_prefix)
# create writer for tensorboard
summary_writer = tf.contrib.summary.create_file_writer(logdir=checkpoint_dir)
print(summary_writer)

./data_out/training_checkpoints\ckpt


In [310]:
EPOCHS = 1000

for epoch in range(EPOCHS):
    
    hidden = encoder.initialize_hidden_state()
    #print(hidden.shape)
    total_loss = 0
    for i, (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
        loss = 0
        #print(s_input, s_input.shape)
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(s_input, hidden)

            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)
            
            #Teacher Forcing: feeding the target as the next input
            for t in range(1, t_input.shape[1]):
                
                predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(t_input[:, t], predictions)
            
                dec_input = tf.expand_dims(t_input[:, t], 1) #using teacher forcing
                
        batch_loss = (loss / int(t_input.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradient = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradient, variables))


    if epoch % 10 == 0:
        #save model every 10 epoch
        print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
                                            total_loss / n_batch,
                                            batch_loss.numpy()))
        #optimizer.minimize(variables)
        checkpoint.save(file_prefix = checkpoint_prefix)


Epoch 0 Loss 0.0994 Batch Loss 2.4839
Epoch 10 Loss 0.0976 Batch Loss 2.4405
Epoch 20 Loss 0.0960 Batch Loss 2.4002
Epoch 30 Loss 0.0945 Batch Loss 2.3615
Epoch 40 Loss 0.0930 Batch Loss 2.3245
Epoch 50 Loss 0.0916 Batch Loss 2.2905
Epoch 60 Loss 0.0903 Batch Loss 2.2578
Epoch 70 Loss 0.0890 Batch Loss 2.2246
Epoch 80 Loss 0.0875 Batch Loss 2.1878
Epoch 90 Loss 0.0859 Batch Loss 2.1478
Epoch 100 Loss 0.0843 Batch Loss 2.1080
Epoch 110 Loss 0.0827 Batch Loss 2.0684
Epoch 120 Loss 0.0811 Batch Loss 2.0275
Epoch 130 Loss 0.0795 Batch Loss 1.9874
Epoch 140 Loss 0.0780 Batch Loss 1.9504
Epoch 150 Loss 0.0766 Batch Loss 1.9145
Epoch 160 Loss 0.0751 Batch Loss 1.8770
Epoch 170 Loss 0.0736 Batch Loss 1.8401
Epoch 180 Loss 0.0721 Batch Loss 1.8034
Epoch 190 Loss 0.0707 Batch Loss 1.7673
Epoch 200 Loss 0.0693 Batch Loss 1.7320
Epoch 210 Loss 0.0679 Batch Loss 1.6971
Epoch 220 Loss 0.0665 Batch Loss 1.6621
Epoch 230 Loss 0.0651 Batch Loss 1.6275
Epoch 240 Loss 0.0637 Batch Loss 1.5931
Epoch 250 L

In [338]:
# #최근 학습된 모델을 불러오고 이어서 EPOCHS로 돌아감
# saver = tf.train.Saver()
# with tf.Session() as sess:
    
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [343]:
#sentence = 'I feel hungry'
sentence1 = """Wounded Afghan veteran Dr. John Watson (Martin Freeman) meets brilliant 
consulting detective Sherlock Holmes (Benedict Cumberbatch) and moves into a flat 
on 221B Baker Street with him. Their landlady, Mrs. Hudson (Una Stubbs), acts as a de 
facto housekeeper for them. Sherlock's first case, "A Study in Pink", sees him apprehend 
cabbi-turned-serial killer Jeff Hope (Phil Davis). Their confrontation escalates to the 
point that John, who's followed Sherlock unnoticed to the scene, shoots the cabbie out of 
fear for Sherlock's life. However, Sherlock manages to extract from the dying cabbie that 
he was being sponsored in his crimes by a figure called Moriarty, who is apparently a "fan" 
of Sherlock's. The case seals Sherlock and John's friendship. In their next case, "The Blind Banker", 
they uncover a Chinese smuggling ring, which is again implied to have been linked to Moriarty.
"The Great Game" sees Sherlock being baited into solving a series of "puzzles" by Moriarty, who is 
as yet unseen. Each puzzle involves solving an undetected crime, with the additional incentive 
being that failure to solve the crime within a set time will lead to the death of an innocent 
bystander. Simultaneously, Sherlock works on recovering the missing Bruce-Partington Plans for 
his older brother Mycroft (Mark Gatiss), who is a powerful figure in the British government. 
The series ends with Sherlock encountering Moriarty (Andrew Scott) by a swimming pool, 
where Moriarty warns Sherlock that he will destroy him."""
sentence2 = """The series ends with Sherlock encountering Moriarty (Andrew Scott) by a swimming pool, 
where Moriarty warns Sherlock that he will destroy him."""
sentence3 = df.movie_summary[200]
print(df.movie_summary[7])
print(df.movie_name[7])
print(df.movie_tagline[7])
#sentence = 'desperado'
#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

henry is resurrected from death with no memory and he must save his wife from a telekinetic warlord with a plan to bioengineer soldiers. waking inside a laboratory on an airship a man recalls bullies from his childhood. a scientist estelle greets him and tells him his name is henry she is his wife and that he has been revived from an accident that left him amnesiac and mute. after she replaces a missing arm and leg with cybernetic prostheses mercenaries led by the psychokinetic akan raid the ship claiming all of estelles research is akans corporate property. he kills estelles scientists before attempting to murder henry but henry and estelle flee in an escape pod landing in moscow. estelle is abducted by the mercenaries who try to kill henry.. hardcore henry is an action film told from a first person perspective you remember nothing. mainly because youve just been brought back from the dead by your wife haley bennett. she tells you that your name is henry. five minutes later you are be

In [340]:
df[df['movie_tagline'].str.contains('dirty mind')]

,level_0,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline,shrinked


In [341]:
sentence3 = """Despite having had several bad experiences while producing meth with Jesse, Walter agrees to rejoin his partner.""".lower()
sentence3

'despite having had several bad experiences while producing meth with jesse, walter agrees to rejoin his partner.'

In [344]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])  
    print(inputs)
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)   
    print(inputs)
    result = ''  
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)     
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
    print(dec_input)
    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += idx2target[predicted_id] + ' '
        print(idx2target[predicted_id])
        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)               
    return result, sentence    
result, output_sentence = prediction(sentence3, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

print(output_sentence)
print(result)

[4616, 237, 4808, 804, 10006]
tf.Tensor(
[[ 4616   237  4808   804 10006     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0]], shape=(1, 60), dtype=int32)
tf.Tensor([[1]], shape=(1, 1), dtype=int32)
it
aint
easy
being
this
good.
<eos>
a family in africa finds itself under attack by a pride of lions.
it aint easy being this good. <eos> 


In [330]:
df[df['movie_tagline'].str.contains('it aint')]

,level_0,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline,shrinked
20,1022,5665,"[Thriller, Crime, Drama]",Night Game,www.imdb.com/title/tt0097971/mediaviewer/rm986...,a police detective tracks a serial killer who ...,"[\n , \n ]",when it comes to murder it aint over till its ...,"[police, detective, tracks, serial, killer, st..."
935,2029,661,"[Comedy, Drama, Sport]",White Men Can't Jump,www.imdb.com/title/tt0105812/mediaviewer/rm403...,black and white basketball hustlers join force...,"[\n , \n ]",it aint easy being this good.,"[black, white, basketball, hustlers, join, for..."


In [334]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])  
    print(inputs)
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)   
    print(inputs)
    result = ''  
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)     
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
    print(dec_input)
    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += idx2target[predicted_id] + ' '
        print(idx2target[predicted_id])
        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)               
    return result, sentence    
result, output_sentence = prediction(sentence1, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

print(output_sentence)
print(result)

[13832, 8129, 3428, 8529, 4881, 132, 3165, 6162, 4836, 11453, 7131, 2613, 4294, 4948, 11681, 1759, 7870, 3925, 1759, 12178, 2937, 4779, 1796, 589, 1939, 11393, 8800, 13496, 11958, 6355, 7539, 11453, 12020, 11534, 14441, 10264, 6716, 12020, 13534, 154, 4589, 12018, 2935, 14285, 11548, 13077, 7374, 3184, 6550, 14338, 10527, 8343, 9057, 1611, 9887, 4779, 11534, 4160, 4139, 12683, 14037, 3413]
tf.Tensor(
[[ 3428  8529  4881   132  3165  6162  4836 11453  7131  2613  4294  4948
  11681  1759  7870  3925  1759 12178  2937  4779  1796   589  1939 11393
   8800 13496 11958  6355  7539 11453 12020 11534 14441 10264  6716 12020
  13534   154  4589 12018  2935 14285 11548 13077  7374  3184  6550 14338
  10527  8343  9057  1611  9887  4779 11534  4160  4139 12683 14037  3413]], shape=(1, 60), dtype=int32)
tf.Tensor([[1]], shape=(1, 1), dtype=int32)
put
christ
back
in
christmas
<eos>
Wounded Afghan veteran Dr. John Watson (Martin Freeman) meets brilliant 
consulting detective Sherlock Holmes (Bened

In [112]:
#len(df.movie_summary[39].split('. '))

In [335]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    print(inputs)
    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    idx = 0
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
    #print(dec_input)
    xList = []
    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)
        #print("*"*100)
        #print(predictions)
        #print("*"*100)
        #print(tf.argmax(predictions[0]))
        print(predictions)
        x = tf.nn.top_k(predictions[0], k=4, sorted=True, name=None)
        for i in range(4):
            predicted_id = x[1][i].numpy()
            #print(predicted_id)

            result += idx2target[predicted_id] + ' '
            #print("result:", result)
            #print(idx2target[predicted_id])
            #if idx2target.get(predicted_id) == '<eos>':
            #             return result, sentence

          #result=""

        # the predicted ID is fed back into the model
            dec_input = tf.expand_dims([predicted_id], 0)  
        xList.append([result])
        result = ''
        
    return xList

# result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
tList=prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
# print(sentence)
from pprint import pprint
pprint(tList)

tf.Tensor(
[[ 3428  8529  4881   132  3165  6162  4836 11453  7131  2613  4294  4948
  11681  1759  7870  3925  1759 12178  2937  4779  1796   589  1939 11393
   8800 13496 11958  6355  7539 11453 12020 11534 14441 10264  6716 12020
  13534   154  4589 12018  2935 14285 11548 13077  7374  3184  6550 14338
  10527  8343  9057  1611  9887  4779 11534  4160  4139 12683 14037  3413]], shape=(1, 60), dtype=int32)
tf.Tensor([[-2.9516444 -2.8085911  0.6763078 ... -2.7793283 -2.8948693 -2.8197725]], shape=(1, 3029), dtype=float32)
tf.Tensor([[-4.235785  -4.330719   2.9213495 ... -4.113395  -3.9793384 -4.0160336]], shape=(1, 3029), dtype=float32)
tf.Tensor([[-3.6485515 -3.2076552  1.3944038 ... -3.3537164 -3.405961  -3.5073025]], shape=(1, 3029), dtype=float32)
tf.Tensor([[-4.1365128 -3.8051496  2.9327059 ... -3.9253533 -4.0261455 -4.1504717]], shape=(1, 3029), dtype=float32)
tf.Tensor([[-5.0449724 -4.9092336  7.1277637 ... -4.735272  -4.8214746 -4.8726535]], shape=(1, 3029), dtype=float32)
tf.